In [43]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from sqlalchemy import create_engine

In [26]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [27]:
# URL of page to be scraped
url = 'https://clinicaltrials.gov/ct2/search/browse?brwse=spns_alpha_all'
browser.visit(url)

In [32]:
try:
    element = browser.find_by_name('theDataTable_length').first
    element.select('-1')
    html = browser.html
    page = pd.read_html(html)
    companystudies_df = pd.DataFrame(page[0], index=None)
except:
    print("Scraping Complete")


In [33]:
companystudies_df.head()

,Sponsor/Collaborators,Studies
0,Ärztliche Zentrum für Qualität in der Medizin,1
1,Åbo Akademi University,5
2,Ångstrom Pharmaceuticals,2
3,Århus Amt,1
4,Çanakkale Onsekiz Mart University,2


In [37]:
companylist_csv = "companylist.csv"
companylist_df = pd.read_csv(companylist_csv)
companylist_df = companylist_df[['Symbol', 'Name', 'LastSale', 'MarketCap', 'IPOyear']].copy()
companylist_df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear
0,YI,"111, Inc.",7.50,5.988416e+07,2018.0
1,MMM,3M Company,185.22,1.067658e+11,NaN
2,AAC,"AAC Holdings, Inc.",1.97,4.984376e+07,2014.0
3,ABT,Abbott Laboratories,78.69,1.382166e+11,NaN
4,ABBV,AbbVie Inc.,78.71,1.163463e+11,2012.0


In [45]:
engine = create_engine('sqlite://', echo=False)
companylist_df.to_sql('company_stock', con=engine, if_exists='replace', index=False)
companystudies_df.to_sql('company_clinicalTrials', con=engine, if_exists='replace', index=False)

In [52]:
pd.read_sql_query("SELECT * FROM company_stock", con=engine).head()

,Symbol,Name,LastSale,MarketCap,IPOyear
0,YI,"111, Inc.",7.50,5.988416e+07,2018.0
1,MMM,3M Company,185.22,1.067658e+11,NaN
2,AAC,"AAC Holdings, Inc.",1.97,4.984376e+07,2014.0
3,ABT,Abbott Laboratories,78.69,1.382166e+11,NaN
4,ABBV,AbbVie Inc.,78.71,1.163463e+11,2012.0


In [53]:
pd.read_sql_query("SELECT * FROM company_clinicalTrials", con=engine).head()

,Sponsor/Collaborators,Studies
0,Ärztliche Zentrum für Qualität in der Medizin,1
1,Åbo Akademi University,5
2,Ångstrom Pharmaceuticals,2
3,Århus Amt,1
4,Çanakkale Onsekiz Mart University,2
